In [1]:
%load_ext bigquery_magics

In [2]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier, early_stopping
from sklearn.metrics import average_precision_score

# "Will it snow tomorrow?" - The time traveler asked
The following dataset contains climate information from over 9000 stations accross the world. The overall goal of these subtasks will be to predict whether it will snow tomorrow 20 years ago. So if today is 1 April 2025 then the weather we want to forecast is for the 2 April 2005. You are supposed to solve the tasks using Big Query, which can be used in the Jupyter Notebook like it is shown in the following cell. For further information and how to use BigQuery in Jupyter Notebook refer to the Google Docs. 

The goal of this test is to test your coding knowledge in Python, BigQuery and Pandas as well as your understanding of Data Science. If you get stuck in the first part, you can use the replacement data provided in the second part.

In [3]:
%%bigquery
SELECT
*,
FROM `bigquery-public-data.samples.gsod`
LIMIT 20 


Query is running:   0%|          |

Downloading:   0%|          |

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,37770,99999,1929,8,4,61.000000,4,53.200001,4,1005.299988,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
1,38640,99999,1929,11,20,52.000000,4,45.200001,4,1004.400024,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
2,33960,99999,1929,10,27,42.500000,4,40.000000,4,1007.500000,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False
3,38940,99999,1929,10,6,54.000000,4,47.500000,4,994.599976,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
4,38110,99999,1929,12,21,44.500000,4,37.500000,4,991.599976,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
5,30910,99999,1929,10,8,43.000000,4,NaN,<NA>,997.500000,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False
6,37950,99999,1929,9,25,56.700001,4,48.500000,4,1031.699951,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False
7,33110,99999,1929,12,20,43.000000,4,38.799999,4,1006.200012,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
8,37950,99999,1929,11,18,35.200001,4,34.000000,4,1017.900024,...,NaN,<NA>,0.00,NaN,True,True,True,True,True,True
9,30750,99999,1929,12,3,44.500000,4,39.700001,4,986.900024,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False


## Part 1

### 1. Task
Change the date format to 'YYYY-MM-DD' and select the data from 2000 till 2005 for station numbers including and between 725300 and 726300 , and save it as a pandas dataframe. Note the maximum year available is 2010. 

In [4]:
%%bigquery df
SELECT DATE(year, month, day) AS date, *
FROM `bigquery-public-data.samples.gsod`
WHERE year BETWEEN 2000 AND 2005
AND station_number BETWEEN 725300 AND 726300


Query is running:   0%|          |

Downloading:   0%|          |

### 2. Task 
From here you want to work with the data from all stations 725300 to 725330 that have information from 2000 till 2005. 

In [5]:
%%bigquery
SELECT *
FROM `bigquery-public-data.samples.gsod`
WHERE year BETWEEN 2000 AND 2005
AND station_number BETWEEN 725300 AND 725330


Query is running:   0%|          |

Downloading:   0%|          |

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,725327,99999,2000,3,17,31.000000,21,20.900000,21,1028.300049,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False
1,725326,99999,2000,12,6,13.700000,22,2.400000,22,NaN,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False
2,725316,99999,2000,5,12,76.800003,23,69.400002,23,NaN,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
3,725317,99999,2000,8,26,70.300003,23,66.400002,23,1013.200012,...,NaN,<NA>,0.13,NaN,False,False,False,False,False,False
4,725305,99999,2000,9,15,55.400002,23,41.799999,23,1019.700012,...,NaN,<NA>,0.02,NaN,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21848,725314,99999,2005,11,10,47.500000,24,26.200001,24,1025.099976,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False
21849,725315,99999,2005,2,25,37.799999,24,30.600000,24,1019.900024,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False
21850,725320,14842,2005,6,11,75.699997,24,67.099998,24,1011.000000,...,NaN,<NA>,0.26,NaN,False,False,False,False,False,False
21851,725305,99999,2005,8,6,72.000000,24,54.299999,24,1021.200012,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False


Start by checking which year received the most snowfall in our data. 

In [6]:
%%bigquery
SELECT year, countif(snow=True) as snowy_days_across_stations, max(snow_depth) as max_snow_depth_across_stations
FROM `bigquery-public-data.samples.gsod`    
WHERE year BETWEEN 2000 AND 2005
AND station_number BETWEEN 725300 AND 725330
GROUP BY year
ORDER BY snowy_days_across_stations DESC

Query is running:   0%|          |

Downloading:   0%|          |

,year,snowy_days_across_stations,max_snow_depth_across_stations
0,2005,826,11.8
1,2000,797,18.1
2,2001,759,15.0
3,2002,745,9.1
4,2003,555,5.9
5,2004,262,7.9


Add an additional field that indicates the daily change in snow depth measured at every station. And identify the station and day for which the snow depth increased the most.  

In [7]:
%%bigquery df
WITH filtered_gsod AS (
    SELECT *, DATE(year, month, day) AS date
    FROM `bigquery-public-data.samples.gsod`
    WHERE year BETWEEN 2000 AND 2005
    AND station_number BETWEEN 725300 AND 725330
),
daily_change_gsod AS ( # For this we can do either a self-join or we use the LAG window function (I'm gonna do both)
    # # --------- LAG Approach ---------
    # SELECT 
    #     station_number, 
    #     date, 
    #     snow_depth - LAG(snow_depth) OVER (PARTITION BY station_number ORDER BY date) AS snow_depth_change,
    #     DATE_DIFF(date, LAG(date) OVER (PARTITION BY station_number ORDER BY date), DAY) AS gap_days
    # FROM filtered_gsod

    # --------- Self-Join Approach ---------
    SELECT gsod1.*, gsod1.snow_depth - gsod2.snow_depth AS snow_depth_change
    FROM filtered_gsod gsod1
    LEFT JOIN filtered_gsod gsod2 ON gsod1.station_number = gsod2.station_number AND gsod2.date = DATE_SUB(gsod1.date, INTERVAL 1 DAY)
)

SELECT *
FROM daily_change_gsod 
ORDER BY snow_depth_change DESC

Query is running:   0%|          |

Downloading:   0%|          |

In [8]:
# station and date of the maximum snow_depth_change
df[df['snow_depth_change'] == max(df['snow_depth_change'])][['station_number', 'date', 'snow_depth_change']]

,station_number,date,snow_depth_change
0,725300,2005-01-22,9.8


Do further checks on the remaining dataset, clean or drop data depending on how you see appropriate. 

In [9]:
df.columns

Index(['station_number', 'wban_number', 'year', 'month', 'day', 'mean_temp',
       'num_mean_temp_samples', 'mean_dew_point', 'num_mean_dew_point_samples',
       'mean_sealevel_pressure', 'num_mean_sealevel_pressure_samples',
       'mean_station_pressure', 'num_mean_station_pressure_samples',
       'mean_visibility', 'num_mean_visibility_samples', 'mean_wind_speed',
       'num_mean_wind_speed_samples', 'max_sustained_wind_speed',
       'max_gust_wind_speed', 'max_temperature', 'max_temperature_explicit',
       'min_temperature', 'min_temperature_explicit', 'total_precipitation',
       'snow_depth', 'fog', 'rain', 'snow', 'hail', 'thunder', 'tornado',
       'date', 'snow_depth_change'],
      dtype='object')

In [10]:
df['date'] = pd.to_datetime(df['date'])

- We need to remove columns where null values represent the majority
- Remove non-relevant feautes
- Impute null values 
- Feature Engineering

In [11]:
# we will drop columns that are about number of samples and explicit measures as they are more 
# about data reliablility which should not be pretty affective for snow prediction

df = df.drop(columns=[
    'wban_number', 
    'year', 
    'month',
    'day'
    ]
)
			
df = df.drop(columns=[
    'num_mean_temp_samples', 
    'num_mean_dew_point_samples', 
    'num_mean_sealevel_pressure_samples',
    'num_mean_station_pressure_samples',
    'num_mean_visibility_samples',
    'num_mean_wind_speed_samples',
    'max_temperature_explicit',
    'min_temperature_explicit',
    ]
)

In [12]:
# Calculate the null percentage for each column 
df.isnull().mean().sort_values(ascending=False)[:13]

min_temperature             1.000000
snow_depth_change           0.966824
snow_depth                  0.960280
mean_station_pressure       0.926829
max_gust_wind_speed         0.433121
mean_sealevel_pressure      0.118382
total_precipitation         0.013545
mean_visibility             0.000732
max_sustained_wind_speed    0.000366
mean_dew_point              0.000320
mean_wind_speed             0.000320
max_temperature             0.000046
mean_temp                   0.000000
dtype: float64

In [13]:
# we will remove the columns with majority nulls as they are useless and will inderduce noise
df = df.drop(columns=[
    'min_temperature', 
    'snow_depth_change', 
    'snow_depth', 
    'mean_station_pressure', 
    'max_gust_wind_speed'
    ]
)

In [14]:
df.columns

Index(['station_number', 'mean_temp', 'mean_dew_point',
       'mean_sealevel_pressure', 'mean_visibility', 'mean_wind_speed',
       'max_sustained_wind_speed', 'max_temperature', 'total_precipitation',
       'fog', 'rain', 'snow', 'hail', 'thunder', 'tornado', 'date'],
      dtype='object')

In [15]:
# Imputing the columns with minority number of nulls using simple median imputation per station
# Because of the temporal nature of the problem, we can also forward fill missing values 
# but weather variables could change drastically from day to day and this is why using the median is safer
median_impute_columns = [
    'mean_sealevel_pressure', 
    'total_precipitation', 
    'mean_visibility', 
    'max_sustained_wind_speed',
    'mean_wind_speed',
    'mean_dew_point',
    'max_temperature'
]
for col in median_impute_columns:
    df[col] = df.groupby(['station_number'])[col].transform(lambda x: x.fillna(x.median()))

/Users/abdullah/Development/7learnings-ds-challenge/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [16]:
df.groupby(['station_number']).agg(missing_percentage=('mean_sealevel_pressure', lambda x: x.isna().mean()))

,missing_percentage
station_number,
725300,0.0
725305,0.0
725314,0.0
725315,0.0
725316,0.0
725317,0.0
725320,0.0
725326,1.0
725327,0.0


In [17]:
# After imputing, there are still missing values for 'mean_sealevel_pressure' and that is because 
# there is one station which never reported this measurement... so I will remove it for the sake of simplicity
df = df[df['station_number'] != 725326]


In [18]:
df.groupby(['station_number']).agg(alltime_snow_percentage=('snow', lambda x: (x == True).mean()))

,alltime_snow_percentage
station_number,
725300,0.374544
725305,0.081772
725314,0.077665
725315,0.084398
725316,0.059442
725317,0.064620
725320,0.447080
725327,0.105774
725330,0.506387


In [19]:
# Each station has some missing dates that we will fill for the sake of computing lags and rolling means in a correct way
df.groupby('station_number').agg(
    max_date=('date', 'max'),
    min_date=('date', 'min'),
    missing_days=('date', lambda x: (
        pd.date_range(start=x.min(), end=x.max()).difference(x).size
    ))
)

,max_date,min_date,missing_days
station_number,,,
725300,2005-12-31,2000-01-01,0
725305,2005-12-31,2000-01-01,3
725314,2005-12-31,2000-01-01,16
725315,2005-12-31,2000-01-01,0
725316,2005-12-31,2000-01-01,5
725317,2005-12-31,2000-01-01,10
725320,2005-12-31,2000-01-01,0
725327,2005-12-31,2000-01-01,27
725330,2005-12-31,2000-01-01,0


In [20]:
df_full = (
    df.set_index(['station_number', 'date'])
      .sort_index()
      .groupby(level=0)
      .apply(lambda group: group.reset_index(level=0, drop=True).resample('D').asfreq())
      .reset_index()
)

In [21]:
df_full.groupby('station_number').agg(
    max_date=('date', 'max'),
    min_date=('date', 'min'),
    missing_days=('date', lambda x: (
        pd.date_range(start=x.min(), end=x.max()).difference(x).size
    ))
)

,max_date,min_date,missing_days
station_number,,,
725300,2005-12-31,2000-01-01,0
725305,2005-12-31,2000-01-01,0
725314,2005-12-31,2000-01-01,0
725315,2005-12-31,2000-01-01,0
725316,2005-12-31,2000-01-01,0
725317,2005-12-31,2000-01-01,0
725320,2005-12-31,2000-01-01,0
725327,2005-12-31,2000-01-01,0
725330,2005-12-31,2000-01-01,0


In [22]:
df_full[df_full['snow'].isna()]

,station_number,date,mean_temp,mean_dew_point,mean_sealevel_pressure,mean_visibility,mean_wind_speed,max_sustained_wind_speed,max_temperature,total_precipitation,fog,rain,snow,hail,thunder,tornado
2660,725305,2001-04-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2725,725305,2001-06-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3865,725305,2004-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4807,725314,2001-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6234,725314,2005-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17191,725327,2005-01-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
17192,725327,2005-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
17193,725327,2005-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
17194,725327,2005-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [23]:
df_full = df_full.sort_values(['station_number', 'date'])


In [24]:
lag_columns = ['snow', 'mean_temp']
for lag_col in lag_columns:
  for lag in range(1, 4):
      df_full[f'{lag_col}_lag_{lag}'] = (
          df_full.groupby('station_number')[lag_col]
            .shift(lag)
      )

In [25]:
df_full["month_sin"] = np.sin(2 * np.pi * df_full["date"].dt.month / 12)
df_full["month_cos"] = np.cos(2 * np.pi * df_full["date"].dt.month / 12)
df_full["doy_sin"] = np.sin(2 * np.pi * df_full["date"].dt.dayofyear / 366)
df_full["doy_cos"] = np.cos(2 * np.pi * df_full["date"].dt.dayofyear / 366)

In [26]:
df_full['snow_tomorrow'] = (
    df_full.groupby('station_number')['snow']
      .shift(-1)   # tomorrow’s snow flag
)

In [27]:
df_full['is_freezing'] = df_full['mean_temp'] <= 32

In [28]:
df_full = df_full[~df_full['snow'].isna()]
df_full = df_full[~df_full['snow_tomorrow'].isna()]

In [29]:
df_full.columns

Index(['station_number', 'date', 'mean_temp', 'mean_dew_point',
       'mean_sealevel_pressure', 'mean_visibility', 'mean_wind_speed',
       'max_sustained_wind_speed', 'max_temperature', 'total_precipitation',
       'fog', 'rain', 'snow', 'hail', 'thunder', 'tornado', 'snow_lag_1',
       'snow_lag_2', 'snow_lag_3', 'mean_temp_lag_1', 'mean_temp_lag_2',
       'mean_temp_lag_3', 'month_sin', 'month_cos', 'doy_sin', 'doy_cos',
       'snow_tomorrow', 'is_freezing'],
      dtype='object')

### 3. Task
Now it is time to split the data, into a training, evaluation and test set. As a reminder, the date we are trying to predict snow fall for should constitute your test set.

In [40]:
import datetime as dt

predict_date = str(dt.datetime.today()- dt.timedelta(days=20*365)).split(' ')[0]


In [41]:
# We want to predict snow for predict_date and this is the why the test date for us will be the day before, as we will be predicting snow for "tomorow"
test_date = (pd.to_datetime(predict_date) - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
df_test = df_full[df_full['date'] == test_date]
df_trainval = df_full[df_full['date'] < test_date]

trainval_dates_sorted = df_trainval['date'].sort_values().unique()
n_val = int(0.1 * len(trainval_dates_sorted))
val_dates = trainval_dates_sorted[-n_val:]
train_dates = trainval_dates_sorted[:-n_val]

df_train = df_trainval[df_trainval['date'].isin(train_dates)]
df_val = df_trainval[df_trainval['date'].isin(val_dates)]

print("Train shape:", df_train.shape)
print("Validation shape:", df_val.shape)
print("Test shape:", df_test.shape)

Train shape: (16673, 28)
Validation shape: (1838, 28)
Test shape: (9, 28)


In [42]:
df_train.groupby(['station_number'])['date'].agg(['min', 'max', 'count'])

,min,max,count
station_number,,,
725300,2000-01-01,2005-02-03,1861
725305,2000-01-01,2005-02-03,1855
725314,2000-01-01,2005-02-03,1857
725315,2000-01-01,2005-02-03,1861
725316,2000-01-01,2005-02-03,1852
725317,2000-01-01,2005-02-03,1846
725320,2000-01-01,2005-02-03,1861
725327,2000-01-01,2005-02-03,1819
725330,2000-01-01,2005-02-03,1861


In [43]:
df_val.groupby(['station_number'])['date'].agg(['min', 'max', 'count'])

,min,max,count
station_number,,,
725300,2005-02-04,2005-08-28,206
725305,2005-02-04,2005-08-28,206
725314,2005-02-04,2005-08-28,190
725315,2005-02-04,2005-08-28,206
725316,2005-02-04,2005-08-28,206
725317,2005-02-04,2005-08-28,206
725320,2005-02-04,2005-08-28,206
725327,2005-02-04,2005-08-28,206
725330,2005-02-04,2005-08-28,206


## Part 2
If you made it up to here all by yourself, you can use your prepared dataset to train an algorithm of your choice to forecast whether it will snow on the following date for each station in this dataset:

In [44]:
import datetime as dt

str(dt.datetime.today() - dt.timedelta(days=20*365)).split(' ')[0]

'2005-08-30'

You are allowed to use any library you are comfortable with such as sklearn, tensorflow, keras etc. 
If you did not manage to finish part one feel free to use the data provided in 'coding_challenge.csv' Note that this data does not represent a solution to Part 1. 

In [45]:
TARGET = "snow_tomorrow"
DROP_COLS = ["date"]
FEATURES = [c for c in df_train.columns if c not in [TARGET] + DROP_COLS]

CATEGORICAL_COLS = ["station_number"]
for df in [df_train, df_val, df_trainval, df_test]:
    if df["station_number"].dtype != "category":
        df["station_number"] = df["station_number"].astype("category")

X_tr, y_tr = df_train[FEATURES], df_train[TARGET]
X_val, y_val = df_val[FEATURES], df_val[TARGET]

pos = max(y_tr.sum(), 1)
neg = max((y_tr == 0).sum(), 1)
scale_pos_weight = neg / pos

lgbm = LGBMClassifier(
    objective="binary",
    n_estimators=10000,
    learning_rate=0.02,
    num_leaves=64,
    max_depth=-1,
    min_child_samples=80,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1,
    verbosity=-1,
    random_state=42
)

lgbm.fit(
    X_tr, 
    y_tr,
    eval_set=[(X_val, y_val)],
    eval_metric="average_precision", 
    categorical_feature=CATEGORICAL_COLS,
    callbacks=[early_stopping(stopping_rounds=200, verbose=0)]
)

train_probs = lgbm.predict_proba(X_tr)[:, 1]
val_probs   = lgbm.predict_proba(X_val)[:, 1]

print("Train PR-AUC:", average_precision_score(y_tr, train_probs))
print("Validation PR-AUC:", average_precision_score(y_val, val_probs))

# Retrain on train+val at best iteration
X_trval, y_trval = df_trainval[FEATURES], df_trainval[TARGET]

lgbm_final = LGBMClassifier(
    objective="binary",
    n_estimators=lgbm.best_iteration_ or 500,
    learning_rate=lgbm.learning_rate,
    num_leaves=lgbm.num_leaves,
    max_depth=lgbm.max_depth,
    min_child_samples=lgbm.min_child_samples,
    subsample=lgbm.subsample,
    colsample_bytree=lgbm.colsample_bytree,
    n_jobs=-1,
    verbosity=-1,
    random_state=42
)

lgbm_final.fit(
    X_trval, 
    y_trval,
    eval_metric="average_precision", 
    categorical_feature=CATEGORICAL_COLS,
)

X_te, y_te = df_test[FEATURES], df_test[TARGET]
test_probs = lgbm_final.predict_proba(X_te)[:, 1]
test_pr_auc = average_precision_score(y_te, test_probs)

print("Test PR-AUC:", test_pr_auc)


/var/folders/1m/g31q_g1x0s7_ld_r4sbn7cp00000gn/T/ipykernel_89343/1514988569.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["station_number"] = df["station_number"].astype("category")


Train PR-AUC: 0.7774088096444304
Validation PR-AUC: 0.5977997649140402
Test PR-AUC: 1.0


In [47]:
pd.DataFrame({
    'station_number': X_te['station_number'].values,
    'snow_tomorrow_truth': y_te.values,
    'snow_tomorrow_pred': test_probs
})


,station_number,snow_tomorrow_truth,snow_tomorrow_pred
0,725300,False,0.371809
1,725305,False,0.063357
2,725314,False,0.107051
3,725315,False,0.249422
4,725316,False,0.102967
5,725317,False,0.257243
6,725320,False,0.355039
7,725327,False,0.052845
8,725330,True,0.736555
